# 1. Geometrical transformations
In these exercises you will first implement Python definitions for computing transformation matrices for different geometrical transformations. Then, you will implement code for converting a transformation matrix to homogeneous form. All needed information for implementing these functions can be found in the lecture slides. In the beginning, you will apply the transformations to geometric objects, however, the same functions will be later used for image transformation.


## 1.1. Transformation matrices
### (A)
Assuming that the coordinates of a 2D geometric object are stored in the variable `X` (the first row contains the horizontal coordinates and the second row contains the vertical coordinates), translation of the geometric object can be performed by adding a 2D translation vector `Xt` to every vertex of `X`:

```python
X[0,:] = X[0,:] + Xt[0]
X[1,:] = X[1,:] + Xt[1]
```

Translation is arguably the "simplest" geometrical transformation that can be applied to an object. Let's leave it aside for now and focus on the other geometrical transformations. The identity, scaling, rotation, reflection and shearing transformations (or any combination of these transformations) can be performed by multiplying the matrix of coordinates `X` with an appropriate transformation matrix `T`. Here is an example of Python code that compute transformation matrices for the identity transformation (which is not really a transformation) and scaling:

```python
import numpy as np

def identity():
    T = np.eye(2)
    return T

def scale(sx, sy):
    T = np.array([[sx,0],[0,sy]])
    return T
```

Using this implementation, scaling can be performed for example in the following way: `X_scaled = scale(2,3)*X`. To verify this, we can use the provided `test_object()` function in the `utilities.py` module that returns a test geometrical object in the shape of the letter *F*, and plot the original object and a scaled version of it (the provided `plot_object()` function in the `utilities.py` module can be used to plot the geometrical object) as follows:

In [ ]:
%matplotlib inline
import sys
sys.path.append("../code")
import numpy as np
import matplotlib.pyplot as plt
import registration as reg
import registration_util as util

X = util.test_object(1)
X_scaled = reg.scale(2, 3).dot(X)

fig = plt.figure(figsize=(5,5))
ax1  = fig.add_subplot(111)
ax1.grid()
util.plot_object(ax1, X)
util.plot_object(ax1, X_scaled)

### EXERCISE:
Implement functions that return transformation matrices for 2D rotation, shearing and reflection. You can find the templates for these three definitions in `# SECTION 1` of the `registration.py` module. To test your implementation, run the `transforms_test()` script from the `registration_tests.py` module and make sure that the output matches the figure below.

<img src="../notebooks/assets/transforms_test.png" width="800" height="600" align="center"/>

In [ ]:
%matplotlib inline
import sys
sys.path.append("../code")
from registration_tests import transforms_test

transforms_test()

### (B)
Geometrical transformations can be combined by multiplying transformation matrices. For example, the following (conceptual) command first applies a 90° rotation to an object and then a vertical reflection: `X_t = reflect(-1,1)*rotate(pi/2)*X`. 

### QUESTION:
Would the result be different if the two transformations in the example above are applied in reversed order? Motivate your answer. 

### EXERCISE:
Test a few more examples of combining transformations. Save the examples in the `combining_transforms()` script template in the `registration_tests.py` module.

In [ ]:
%matplotlib inline
import sys
sys.path.append("../code")
from registration_tests import combining_transforms

combining_transforms()

## 1.2. Homogeneous coordinates
### (A)
As it was mentioned at the beginning of the previous exercise, translation can be performed by adding a translation vector to the coordinates of an object. Translation can be combined with other geometrical transformations, for example:

In [ ]:
%matplotlib inline
import sys
sys.path.append("../code")
import numpy as np
import matplotlib.pyplot as plt
import registration as reg
import registration_util as util

X = util.test_object(1)

# translation vector
t = np.array([10, 20])

# rotate by 45 deg.
X_rot = reg.rotate(np.pi/4).dot(X)

# translate by 10 in the horizontal and 20 in the vertical direction
X_rot_tran = np.empty(shape=X.shape)
X_rot_tran[0,:] = X_rot[0,:] + t[0];
X_rot_tran[1,:] = X_rot[1,:] + t[1];

fig = plt.figure(figsize=(5,5))
ax1  = fig.add_subplot(111)
ax1.grid()
util.plot_object(ax1, X)
util.plot_object(ax1, X_rot_tran)

However, this way of combining translation with other transformations can be a bit cumbersome (it somewhat complicates the mathematical notation and implementation in code). The transformations that you have implemented in the previous exercise can be straightforwardly combined with translation by converting the transformation matrix to homogeneous form. This matrix can then be applied to the homogeneous coordinates (details can be found in the lecture slides). The function `c2h()` given below (also available in the `utilities.py` module) implements conversion from Cartesian coordinates to homogeneous coordinates. As you can see from the code, this conversion is performed by simply adding an additional row of coordinates with all ones:

In [ ]:
%matplotlib inline
import sys
sys.path.append("../code")
import registration_util as util
import numpy as np

X = util.test_object(1)
Xh = util.c2h(X)

print('X:\n{}\n'.format(X))
print('Xh:\n{}\n'.format(Xh))

### EXERCISE:
Implement the function called `t2h()` in the `utilities.py` module that converts a transformation matrix and a translation vector to a transformation matrix in homogeneous form. The template for this definition is already provided in the module file. To test your function, verify that the `t2h_test()` script results in the same object as the example above (note that the function `plot_object()` also works with homogeneous coordinates):

In [ ]:
%matplotlib inline
import sys
sys.path.append("../code")
from registration_tests import t2h_test

t2h_test()

### (B)
The rotation transformation rotates the objects counterclockwise around the origin of the coordinate system. To perform rotation around an arbitrary point, the following sequence of transformations must be applied:
1. Translate the object so the arbitrary rotation point is translated to the origin of the coordinate system
2. Rotate the object
3. Translate the object back so that the arbitrary rotation point is in the original location.

These three transformation can be combined by multiplying the three homogeneous transformation matrices. Combining transformation matrices in homogeneous form works in the same way as the "regular" transformation matrices, i.e. by matrix multiplication. 

### EXERCISE:
Write an example that rotates the test object by 45° around the first vertex (hint: the first vertex is `X[:,0]` and `t2h(reg.identity(), Xt)` is a homogeneous transformation matrix that performs only translation). Save the example in the provided `arbitrary_rotation()` template in the `registration_tests.py` module. The result should match the one shown in the figure below.

<img src="../notebooks/assets/arbitrary_rotation.png" width="400" align="center"/>


In [ ]:
%matplotlib inline
import sys
sys.path.append("../code")
from registration_tests import arbitrary_rotation

arbitrary_rotation()

### (C)
### QUESTION:
What is rigid and what affine transformation? How many degrees of freedom dothese two types of transformations have in 2D?